In [4]:
from pymodbus.client import ModbusTcpClient
import socket
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadDecoder

In [5]:
ip = '192.168.0.113'
port = 502

register_address_map = {"TEMP_PV": 0,
                        "RH_PV": 2,
                        "RH_SV_Read_Only": 3,
                        "Program_Status": 4,
                        "TEMP_SV": 7,
                        "RH_SV_Read_Write": 16
                        } 
coil_address_map = {"Profile_Run": 0,
                    "Lamp_Command": 1,
                    "Manual_Run" : 2,
                    "Batch_Start_Flag": 3
                   }

In [7]:
client = ModbusTcpClient(host=ip, port=port)
connection = client.connect()

Connection to (192.168.0.113, 502) failed: timed out


In [4]:
while True:
    try:
        for key, value in coil_address_map.items():
            rd = client.read_coils(value)
            print()
            print(rd)
            print(f"{key: <10}: {rd.bits}")
        break
    except Exception as e:
        pass


ReadCoilsResponse(8)
Profile_Run: [False, False, False, False, False, False, False, False]

ReadCoilsResponse(8)
Lamp_Command: [False, False, False, False, False, False, False, False]

ReadCoilsResponse(8)
Manual_Run: [False, False, False, False, False, False, False, False]

ReadCoilsResponse(8)
Batch_Start_Flag: [True, True, True, True, True, True, True, True]


In [11]:
try:
    client.write_coil(1, True)
except:
    None

In [6]:
while True:
    try:
        rd = client.read_holding_registers(16, 2)
        print()
        assert(rd.function_code < 0x80)     # test that we are not an error
        print(rd)
        print(rd.registers)
        break
    except:
        None


ReadHoldingRegistersResponse (2)
[0, 17008]


In [7]:
while True:
    try:
        registers = rd.registers
# decoder = BinaryPayloadDecoder.fromRegisters(registers, wordorder=Endian.LITTLE)
        decoder = BinaryPayloadDecoder.fromRegisters(registers, Endian.BIG, wordorder=Endian.LITTLE)
        print ("Result: " + str(decoder.decode_32bit_float()))
        break
    except:
        pass

Result: 60.0


In [8]:
rd.registers[1]*100/65535

25.952544441901274

In [9]:
while True:
    try:
        rq = client.write_register(16, 50)       # RH_SV_Read_Write = 70 %
        rd = client.read_holding_registers(value)
        print(rd)
        break
    except:
        pass

ReadHoldingRegistersResponse (1)


In [10]:
print("\nReading holding registers:")
while True:
    try:
        for key, value in register_address_map.items():
            rd = client.read_holding_registers(value)
            print()
            print(rd)
            print(f"{key: <10}: {rd.registers}")
        break
    except:
        None


Reading holding registers:

ReadHoldingRegistersResponse (1)
TEMP_PV   : [247]

ReadHoldingRegistersResponse (1)
RH_PV     : [95]

ReadHoldingRegistersResponse (1)
RH_SV_Read_Only: [60]

ReadHoldingRegistersResponse (1)
Program_Status: [0]

ReadHoldingRegistersResponse (1)
TEMP_SV   : [250]

Exception Response(131, 3, IllegalAddress)
RH_SV_Read_Write: []


In [ ]:
print("\nWriting to set value holding registers:\n")

tsv=int(input("Write TEMP_SV in C"))
rhsv=int(input("Write RH_SV"))

while True:
    try:
        key = "TEMP_SV"
        value = register_address_map[key]
        rq = client.write_register(value, tsv*10)       # TEMP_SV = 20.0 deg C
        rd = client.read_holding_registers(value)
        print(rd)
        print(f"{key: <10}: {rd.registers}")
        rhsv_binary = format(rhsv, '032b')

        rhsv_high_binary = rhsv_binary[:16]
        rhsv_low_binary = rhsv_binary[16:]
        rhsv_high = int(rhsv_high_binary, 2)
        rhsv_low = int(rhsv_low_binary, 2)
        key = "RH_SV_Read_Write"
        value_l=register_address_map[key]
        value_h=value_l+1
        
        rq_l=client.write_register(value_l, rhsv_low)
        rd_l=client.read_holding_registers(value_l)
        rq_h=client.write_register(value_h, rhsv_high)
        rd_h=client.write_register(value_h)
        
        break
    except:
        pass


Writing to set value holding registers:



Write TEMP_SV in C 25
Write RH_SV 50


In [ ]:
try:
    client.write_coil(2, True)
except:
    None

In [ ]:
client.close()

In [14]:
rd=client.read_holding_registers(0)
tc_temp=float(rd.registers)

Connection to (192.168.0.113, 502) failed: timed out


ConnectionException: Modbus Error: [Connection] Failed to connect[ModbusTcpClient 192.168.0.113:502]